In [16]:
import numpy as np

import matplotlib.pyplot as plt
from numpy import linalg
from random import random
from copy import deepcopy

In [17]:
X = np.reshape([random()*100 for i in range(400)],(100,4))

In [18]:
col_max = np.amax(X,axis=0)
col_min = np.amin(X,axis=0)
centroid = (col_max + col_min)/2

In [19]:
Z = (X-centroid)**2

In [20]:
alpha = (1.0 / 100) * np.ones(100)

In [21]:
ZT = Z.T
h = np.matmul(alpha,Z)
h

array([762.06456943, 762.48833024, 827.30172556, 943.58890861])

In [22]:
def isclose(a, b, rel_tol, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [23]:
isclose(1.00,1.002,1e-02)

True

In [24]:
def calc_s(h,alpha_old,Z,u,i):
  ans=0
  for j in range(4):
            s_i_j = h[j] - alpha_old[i]*Z[i][j]
            ans += (1/(u + s_i_j/Z[i][j]))
  return ans

In [25]:
def binary(h,alpha_old,Z,i,low,high):
  mid=(low+high)/2
  temp_ans=calc_s(h,alpha_old,Z,mid,i)
  while(not isclose(temp_ans,1,1e-05)):
    # print(abs(temp_ans-1))
    if(temp_ans>1):
      low=mid
    elif(temp_ans<1):
      high=mid
    else:
      return mid
    mid = (low+high)/2
    # print(low,high,mid)
    temp_ans=calc_s(h,alpha_old,Z,mid,i)
  return mid

In [26]:
def helper(h,alpha_old,Z,u,i):

    u_old=u # this 0.00001 is step which i am decreasing the u for binary search..
    
    while(calc_s(h,alpha_old,Z,u,i)>1):
        u_old=u
        u*=2        
    if u_old == u:
        u = binary(h,alpha_old,Z,i,0,1)
    else:
        u =binary(h,alpha_old,Z,i,u_old,u)
    return u

In [27]:
def solution(Z,alpha,h,tol):
    i = 0
    updates = 0
    converged = False
    alpha_old = np.zeros(100)
    old = 0
        
    while i < 100:
        if converged:
            return h
        alpha_old[i] = alpha[i]
        f_x = 0
        u=0
        f_x=calc_s(h,alpha_old,Z,u,i)
        
        if f_x == 1:
            alpha[i] = u
        
        if f_x < 1:

            alpha[i] = 0

        if f_x > 1:
            alpha[i]= helper(h,alpha_old,Z,1,i)
            
                
        h = h + (alpha[i] - alpha_old[i])*Z[i]
        i += 1
        if i>=99 :
            i = 0
            updates += 1
            if old == 0:
                stored_alpha_old = deepcopy(alpha_old)
                old = 1
        if updates == 4:
            if np.all(alpha - stored_alpha_old) < tol :  #.00001
                converged = True
            else:
                updates = 0
                stored_alpha_old = deepcopy(alpha)
    

In [28]:
h = solution(Z,alpha,h,1e-05)
h

array([9144.37935994, 5063.04084804, 8433.10957782, 7448.28805468])

## weight vectors obtained are:

In [29]:
v = 1/h
v

array([0.00010936, 0.00019751, 0.00011858, 0.00013426])

## visualising the ellipsoid with data points
